# DATABASE TUTORIAL WITH : py_to_psql & opendtect_to_py 

In [1]:
# Base environment
import time 
import os
# Core libraries
import psycopg2 as p
import odpy.wellman as wm
import pandas as pd
import numpy as np
# Own
import py_to_psql as pp
import opendtect_to_py as op

# Setting the stage
---
Before accessing Opendtect database and run any workflow, few steps are needed:
1. Connect to the remote database
2. Create tables to work with
3. Fetch available wells in the Opendtect Survey

## Database connection
Any database work, local or remote, requires a computer connection to the database. The connection is similar to the "with open" flow to work with documents. Therefore, the first step will always be to declare the connection to the remote database.

In [2]:
conn = p.connect(
    host="192.168.1.107",
    database="mnsh_thesis",
    user="lsdr94",
    password="Pelusa120894"
)

## Fetch well names
Most of Opendtect methods works by well. Therefore fetch well names inside Opendtect's survey database is a must. Can be done manually using odpy.wellman.getNames method. Most methods of the wellman's module are time consuming; this is why Opendtect database must be consulted once for any desired data. Beware of this fact. 

In [3]:
well_names = wm.getNames(reload=True)

## Table creation
While connected to a database, tables can be created through ***py_to_psql.wells_table_creation*** function. By default creates a basic table (see the function's docstring) to store overall information of wells stored in Opendtect's survey database. For a more complex table creation, the user has two options:
1. Provide the desired column configuration to ***py_to_psql.wells_table_creation*** function through it's column_list argument.
2. Create a query from scratch and execute it with ***py_to_psql.execute_psql_command*** function.

The next cell illustrates the first option. If tables already exist in the database, an error will rise.

In [3]:
# Overall info: wells
pp.wells_table_creation("wells", conn)

# Track table
track_cols = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "md_in_m NUMERIC(10,2)[] NOT NULL,",
    "tvdss_in_m NUMERIC(10,2)[] NOT NULL,",
    "x_coordinate NUMERIC(10,2)[] NOT NULL,",
    "y_coordinate NUMERIC(10,2)[] NOT NULL,",
    "log_name VARCHAR(30)"
]
pp.wells_table_creation("tracks", conn, column_list=track_cols)

# Checkshot table
checkshot_cols = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "md_in_m NUMERIC(13,5)[],",
    "owt_in_ms NUMERIC(13,5)[],",
    "log_name VARCHAR(40)"
]
pp.wells_table_creation("checkshots", conn, column_list=checkshot_cols)

# gr table
gr_cols = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "md_in_m NUMERIC(13,5)[],",
    "gr NUMERIC(13,5)[],",
    "log_name VARCHAR(30)"
]
pp.wells_table_creation("gr_table", conn, column_list=gr_cols)

# compressional wave table
dtco_cols = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "md_in_m NUMERIC(13,5)[],",
    "dtco NUMERIC(13,5)[],",
    "log_name VARCHAR(30)"
]
pp.wells_table_creation("dtco_table", conn, column_list=dtco_cols)

# shear wave table
dtsh_cols = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "md_in_m NUMERIC(13,5)[],",
    "dtsh NUMERIC(13,5)[],",
    "log_name VARCHAR(30)"
]
pp.wells_table_creation("dtsh_table", conn, column_list=dtsh_cols)

# porosity table
porosity_cols = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "md_in_m NUMERIC(13,5)[],",
    "nphi NUMERIC(13,5)[],",
    "log_name VARCHAR(30)"
]
pp.wells_table_creation("nphi_table", conn, column_list=porosity_cols)

# density table
density_cols = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "md_in_m NUMERIC(15,7)[],",
    "rhob NUMERIC(15,7)[],",
    "log_name VARCHAR(30)"
]
pp.wells_table_creation("rhob_table", conn, column_list=density_cols)

'Query has been executed successfully in 0.11027407646179199s'

# Fetch and insert Opendtect's overall well info
---
Most of Opendtect data can be fetch as python dictionaries using wellman module. But still, the process of accessing this data can be time consuming. This is one of the reasons to migrate data into PSQL databases. All the basic informatión can be stored in an iterator of dictionaries using ***opendtect_to_py.fetch_opendtect_wells_info*** function. This iterator ensures to drop dictionaries after using them in order to preserve memmory consuption.

In [8]:
opdt_wells = op.fetch_opendtect_wells_info()

In [9]:
print(opdt_wells)
# Print objects
for well in opdt_wells:
    print(well)
    # Too many wells. Lets break the loop.
    break
opdt_wells = op.fetch_opendtect_wells_info()

{'ID': '100050.3', 'Name': '19/08- 1', 'Format': 'dGB', 'TranslatorGroup': 'Well', 'File_name': 'C:\\Users\\Monster\\Documents\\Opendtect\\MNSH_OGA_2016_v1\\WellInfo\\19_08-_1.well', 'Status': 'OK', 'X': 236383.46063858, 'Y': 6405003.51221513}


## Insert well information in table
To insert data into tables, an insertion query must be created. It constitutes of a series of formatted strings and values that PSQL interprets. The following cell illustrates the process of query creation and insertion well by well. The function ***py_to_psql.string_replacement*** allows to quickly drop symbols that normally PSQL can not interpret as text or commands; thefore, it will be used to make rapid modifications to python strings. The ***py_to_psql.execute_psql_command*** function constitutes the one of the main functions to populate data into PSQL tables. It ensures flexibility when working with tables. Users can execute any queries as long as they are well written.     
    
Pay attention to how slow the process is VS the amount of seconds needed to proccess the insertion query. The slowness of the process is due to Opendtect's limitation mention before.



In [10]:
init = time.time()

table_name = "wells"
cols = ["opendtect_id", "well_name", "x_coordinate", "y_coordinate", "status"]
    
for well_dict in opdt_wells:
    init2 = time.time()
    well_upsertion_query = f"""
        INSERT INTO {table_name}({pp.string_replacement(str(cols))})
        VALUES ({well_dict['ID']}, '{well_dict['Name']}', {well_dict['X']}, {well_dict['Y']}, '{well_dict['Status']}')
        ON CONFLICT (well_name) DO NOTHING;
    """      
    pp.execute_psql_command(well_upsertion_query, conn)
    end2 = time.time()
    print(f"Proccessing insertion query. Concept: well {well_dict['Name']}'s general info in {end2 - init2}s")
end = time.time()
print(f"Wells insertion completed in {end - init} s")

Proccessing insertion query. Concept: well 19/08- 1's general info in 0.047875165939331055s
Proccessing insertion query. Concept: well 19/15- 1's general info in 0.019240140914916992s
Proccessing insertion query. Concept: well 20/06- 1's general info in 0.018945932388305664s
Proccessing insertion query. Concept: well 20/06- 2's general info in 0.020021915435791016s
Proccessing insertion query. Concept: well 20/06- 3's general info in 0.025109291076660156s
Proccessing insertion query. Concept: well 20/06- 3Z's general info in 0.010458230972290039s
Proccessing insertion query. Concept: well 20/06- 6's general info in 0.0217587947845459s
Proccessing insertion query. Concept: well 20/12- 1's general info in 0.021225929260253906s
Proccessing insertion query. Concept: well 20/12- 2's general info in 0.043129920959472656s
Proccessing insertion query. Concept: well 20/12- 3's general info in 0.028194189071655273s
Proccessing insertion query. Concept: well 21/21- 1's general info in 0.007110357

Note that the creation of queries using python is flexible and has a lot of potential but is rather anonnying. This repetitive process with many variables can be confusing. Hence, functions to construct specific queries for data population are provided.    
    
***py_to_psql.string_replacement*** uses the REPLACE_DICT class variable. If the function does not provide an useful result, try adding to this dictionary a pair of key and value to work on.

In [15]:
pp.REPLACE_DICT

{'[': '',
 "'": '',
 ', ': ',',
 ',  ': ',',
 ' ': '_',
 '-': '_',
 '/': '_',
 '`': '_',
 ']': ''}

# Fetch database data
---
The opposite process of insertion is consultation. Although ***py_to_psql.execute_psql_command*** executes queries, it doesn't returns any data. To fetch data from tables the ***py_to_psql.fetch_psql_command*** function must be invoked. The following cell illustrates the format of the fetched data. Note that the first list refers to the column names of our most recently worked table.

In [13]:
query = "SELECT * FROM wells LIMIT 1"
pp.fetch_psql_command(query, conn)

(['well_id',
  'opendtect_id',
  'well_name',
  'x_coordinate',
  'y_coordinate',
  'status'],
 [(1,
   Decimal('100050.3'),
   '19/08- 1',
   Decimal('236383.46'),
   Decimal('6405003.51'),
   'OK')])

## Fetch column names of any table
To quickly access any table's column names, ***py_to_psql.fetch_column_names*** must be used

In [14]:
pp.fetch_column_names("wells", conn)

['well_id',
 'opendtect_id',
 'well_name',
 'x_coordinate',
 'y_coordinate',
 'status']

In [17]:
# Illustration of string replacement regarding the column names
pp.string_replacement(str(pp.fetch_column_names("wells", conn)))

'well_id,opendtect_id,well_name,x_coordinate,y_coordinate,status'

# Fetch Opendtect logs
---
As it was mention before, most of the functions are designed to handle one well at a time. Fetching an specific log follows this assumption. Thus the function ***op.fetch_opendtect_well_log*** must be fed with well and log names. The *odpy.wellman* module will rise and error if there is no well or log in Opendtect's database.    
   
Note again the format of the fetched data from Opendtect. In this case, it is a tuple with two list: one for MD and the other one corresponds with the measurement related to the actual log. Both list have the same lenght, and a pair elements between both list compound a ***sample***    
    
In further functions, these lists that constitute the log will be stored as such, a list, into PSQL tables. This approach ensures that a table contains all logs of the same nature sorted by wells, rather than having N tables for each well. This will be shown in further detail in the DataFrame section

In [18]:
op.fetch_opendtect_well_log("19/08- 1", "Joined Well Logs`DTCO [D]")

([297.78961181640625,
  297.9419860839844,
  298.0943908691406,
  298.2467956542969,
  298.3992004394531,
  298.5516052246094,
  298.70401,
  298.8564147949219,
  299.0087890625,
  299.16119384765625,
  299.3135986328125,
  299.46600341796875,
  299.618408203125,
  299.77081298828125,
  299.9231872558594,
  300.0755920410156,
  300.2279968261719,
  300.3804016113281,
  300.5328063964844,
  300.6852111816406,
  300.83758544921875,
  300.99,
  301.14239501953125,
  301.2947998046875,
  301.44720458984375,
  301.599609375,
  301.75201416015625,
  301.9043884277344,
  302.0567932128906,
  302.2091979980469,
  302.3616027832031,
  302.5140075683594,
  302.6664123535156,
  302.81878662109375,
  302.97119140625,
  303.12359619140625,
  303.276,
  303.42840576171875,
  303.580810546875,
  303.7331848144531,
  303.8855895996094,
  304.0379943847656,
  304.1903991699219,
  304.3428039550781,
  304.4952087402344,
  304.6476135253906,
  304.8,
  304.952392578125,
  305.10479736328125,
  305.257202

## Log insertion
As mentioned before, some functions provide an automatization process to populate data into tables. This is the case with ***opendtect_to_psql.insert_logs*** function, which uses the fetched well names as an argument. The following cells contains the invocation of this function to insert data of *track*, *checkshot*, *gr*, *dtco*, *dtsh*, *rhob* & *nphi* into their corresponding table.    
    
If no log is detected for a given well, an error will be rised but the process will continue.    
     
Log names can be searched manually or using *odpy.wellman.Get*


In [27]:
wm.getLogNames('19/15- 1')

['Joined Well Logs`Resistivity`P16H_UNC',
 'Joined Well Logs`Resistivity`P22H_UNC',
 'Joined Well Logs`Resistivity`P28H_UNC',
 'Joined Well Logs`Resistivity`P34H_UNC',
 'Joined Well Logs`Resistivity`P40H_UNC',
 'Joined Well Logs`GR',
 'Joined Well Logs`DTCO [D]',
 'Joined Well Logs`Input to Derived Logs`DT']

In [20]:
print("*"*25, "Track table", "*"*25)
op.insert_logs(well_names, "track", "tracks", "wells", conn)

************************* Track table *************************
Proccessing insertion query. Concept: well log 'track' insertion in array mode

Well 19/08- 1

Well 19/15- 1

Well 20/06- 1

Well 20/06- 2

Well 20/06- 3

Well 20/06- 3Z

Well 20/06- 6

Well 20/12- 1

Well 20/12- 2

Well 20/12- 3

Well 21/21- 1

Well 21/26- 1D

Well 26/04- 1

Well 26/07- 1

Well 26/08- 1

Well 26/12- 1

Well 26/14- 1

Well 27/03- 1

Well 27/10- 1

Well 28/02a- 2

Well 28/04a- 3

Well 28/12- 1

Well 28/15- 1

Well 29/06b- 5

Well 29/11a- 1

Well 29/12- 1

Well 29/16- 1

Well 29/18- 1

Well 29/19- 1A

Well 29/19- 2

Well 29/20- 1

Well 29/23- 1

Well 29/23b- 2

Well 29/24- 1

Well 29/27- 1

Well 30/21- 1

Well 30/23- 1

Well 30/27- 1

Well 30/28- 1

Well 30/28- 2

Well 30/29a- 3

Well 30/30- 3

Well 31/26a- 12

Well 31/26c- 13

Well 31/27- 1

Well 36/13- 1

Well 36/15- 1

Well 36/26- 1

Well 37/10- 1

Well 37/12- 1

Well 37/23- 1

Well 37/25- 1

Well 38/01- 1

Well 38/03- 1

Well 38/10- 1

Well 38/16- 1

Wel

"\nLog 'track' insertion completed in 239.245192527771s"

In [21]:
print("*"*25, "Checkshot table", "*"*25)
op.insert_logs(well_names, "Checkshots`One-way time 1[active]", "checkshots", "wells", conn)

************************* Checkshot table *************************
Proccessing insertion query. Concept: well log 'Checkshots`One-way time 1[active]' insertion in array mode

Well 19/08- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.3', 'Checkshots`One-way time 1[active]']' returned non-zero exit status 1.
Log Checkshots`One-way time 1[active] not found for Well 19/08- 1.

Well 19/15- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Checkshots`One-way time 1[active]']' returned non-zero exit status 1.
Log Checkshots`One-way time 1[active] not found for Well 19/15- 1.

Well 20/06- 1

Well 20/06- 2

Well 20/06- 3

Well 20/06- 3Z

Well 20/06- 6

Well 20/12- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.19', 'Checkshots`One-way time 1[active]']' returned non-zero exit status 1.
Log Chec

"\nLog 'Checkshots`One-way time 1[active]' insertion completed in 261.15387535095215s"

In [22]:
print("*"*25, "GR table", "*"*25)
op.insert_logs(well_names, "Joined Well Logs`GR", "gr_table", "wells", conn)

************************* GR table *************************
Proccessing insertion query. Concept: well log 'Joined Well Logs`GR' insertion in array mode

Well 19/08- 1

Well 19/15- 1

Well 20/06- 1

Well 20/06- 2

Well 20/06- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Joined Well Logs`GR']' returned non-zero exit status 1.
Log Joined Well Logs`GR not found for Well 20/06- 3.

Well 20/06- 3Z
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.217', 'Joined Well Logs`GR']' returned non-zero exit status 1.
Log Joined Well Logs`GR not found for Well 20/06- 3Z.

Well 20/06- 6

Well 20/12- 1

Well 20/12- 2

Well 20/12- 3

Well 21/21- 1

Well 21/26- 1D
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.221', 'Joined Well Logs`GR']' returned non-zero exit status 1.
Log Joined Well Logs`GR not foun

"\nLog 'Joined Well Logs`GR' insertion completed in 295.77659916877747s"

In [23]:
print("*"*25, "Compressional wave table", "*"*25)
op.insert_logs(well_names, "Joined Well Logs`DTCO [D]", "dtco_table", "wells", conn)

************************* Compressional wave table *************************
Proccessing insertion query. Concept: well log 'Joined Well Logs`DTCO [D]' insertion in array mode

Well 19/08- 1

Well 19/15- 1

Well 20/06- 1

Well 20/06- 2

Well 20/06- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Joined Well Logs`DTCO [D]']' returned non-zero exit status 1.
Log Joined Well Logs`DTCO [D] not found for Well 20/06- 3.

Well 20/06- 3Z
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.217', 'Joined Well Logs`DTCO [D]']' returned non-zero exit status 1.
Log Joined Well Logs`DTCO [D] not found for Well 20/06- 3Z.

Well 20/06- 6

Well 20/12- 1

Well 20/12- 2

Well 20/12- 3

Well 21/21- 1

Well 21/26- 1D
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.221', 'Joined Well Logs`DTCO [D]']' returned non-

"\nLog 'Joined Well Logs`DTCO [D]' insertion completed in 304.8377249240875s"

In [24]:
print("*"*25, "Shear wave table", "*"*25)
op.insert_logs(well_names, "Joined Well Logs`DTSH [D]", "dtsh_table", "wells", conn)

************************* Shear wave table *************************
Proccessing insertion query. Concept: well log 'Joined Well Logs`DTSH [D]' insertion in array mode

Well 19/08- 1

Well 19/15- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Joined Well Logs`DTSH [D]']' returned non-zero exit status 1.
Log Joined Well Logs`DTSH [D] not found for Well 19/15- 1.

Well 20/06- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.7', 'Joined Well Logs`DTSH [D]']' returned non-zero exit status 1.
Log Joined Well Logs`DTSH [D] not found for Well 20/06- 1.

Well 20/06- 2
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.11', 'Joined Well Logs`DTSH [D]']' returned non-zero exit status 1.
Log Joined Well Logs`DTSH [D] not found for Well 20/06- 2.

Well 20/06- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1

"\nLog 'Joined Well Logs`DTSH [D]' insertion completed in 313.82961654663086s"

In [25]:
print("*"*25, "Porosity table", "*"*25)
op.insert_logs(well_names, "Joined Well Logs`NPHI [D]", "nphi_table", "wells", conn)

************************* Porosity table *************************
Proccessing insertion query. Concept: well log 'Joined Well Logs`NPHI [D]' insertion in array mode

Well 19/08- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.3', 'Joined Well Logs`NPHI [D]']' returned non-zero exit status 1.
Log Joined Well Logs`NPHI [D] not found for Well 19/08- 1.

Well 19/15- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Joined Well Logs`NPHI [D]']' returned non-zero exit status 1.
Log Joined Well Logs`NPHI [D] not found for Well 19/15- 1.

Well 20/06- 1

Well 20/06- 2

Well 20/06- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Joined Well Logs`NPHI [D]']' returned non-zero exit status 1.
Log Joined Well Logs`NPHI [D] not found for Well 20/06- 3.

Well 20/06- 3Z
Failed:  Command '['C:\\PRO

"\nLog 'Joined Well Logs`NPHI [D]' insertion completed in 313.71649646759033s"

In [26]:
print("*"*25, "Density table", "*"*25)
op.insert_logs(well_names, "Joined Well Logs`RHOB", "rhob_table", "wells", conn)

************************* Density table *************************
Proccessing insertion query. Concept: well log 'Joined Well Logs`RHOB' insertion in array mode

Well 19/08- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.3', 'Joined Well Logs`RHOB']' returned non-zero exit status 1.
Log Joined Well Logs`RHOB not found for Well 19/08- 1.

Well 19/15- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Joined Well Logs`RHOB']' returned non-zero exit status 1.
Log Joined Well Logs`RHOB not found for Well 19/15- 1.

Well 20/06- 1

Well 20/06- 2

Well 20/06- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Joined Well Logs`RHOB']' returned non-zero exit status 1.
Log Joined Well Logs`RHOB not found for Well 20/06- 3.

Well 20/06- 3Z
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\

"\nLog 'Joined Well Logs`RHOB' insertion completed in 314.48315024375916s"

## Table completion
From the previous outputs, there are some wells that doesn't contain the provided log. Some of these have logs with other names, therefore look for them is a must. The function ***opendtect_to_py.check_null_wells*** allow the user to quickly check all the logs under the wells that raised errors in the previous interaction also whether the well have any related log or not. In this context, empty wells will be those that do not present any record and null wells those that contains logs but not the chosen one. Empty is a subset of nulls.

In [42]:
gr_null_wells, gr_empty_wells = op.check_null_wells("gr", "gr_table", "wells", conn)


Well '20/06- 3' logs:
****************************************
Checkshots`One-way time 1[active]

Well '20/06- 3Z' logs:
****************************************
Checkshots`One-way time 1[active]

Well '29/06b- 5' logs:
****************************************
Checkshots`One-way time 1[active]

Well '30/28- 2' logs:
****************************************
Raw CDA Logs`DTCO [D]
Raw CDA Logs`Input to Dervied Logs`DTCO1
Raw CDA Logs`Input to Dervied Logs`DTSM1
Raw CDA Logs`DTSH [D]

Well '37/12- 1' logs:
****************************************
Joined Well Logs`RHOB
Joined Well Logs`Deep Resistivity`ILD
Joined Well Logs`SP
Joined Well Logs`Medium Resistivity`ILM
Joined Well Logs`Micro Resistivity`SFLU
Joined Well Logs`CALI
Joined Well Logs`DRHO
Joined Well Logs`Input to Derived Logs`DT
Joined Well Logs`Input to Derived Logs`NPHI
Joined Well Logs`Input to Derived Logs`PEF
Joined Well Logs`DTCO [D]
Joined Well Logs`NPHI [D]
Joined Well Logs`PEF [D]

Well '39/01- 2' logs:
*****************

In [43]:
print("Null Wells: \n", gr_null_wells)
print("\nEmpty Wells: \n",gr_empty_wells)

Null Wells: 
 ['20/06- 3', '20/06- 3Z', '21/26- 1D', '28/04a- 3', '28/15- 1', '29/06b- 5', '29/19- 1A', '30/28- 2', '30/29a- 3', '37/12- 1', '39/01- 2', '39/02- 1', '41/01- 1', '41/18- 1', '41/18- 2', '41/20- 1', '41/25a- 1', '42/16- 1', '43/10- 1']

Empty Wells: 
 ['21/26- 1D', '28/04a- 3', '28/15- 1', '29/19- 1A', '30/29a- 3', '39/02- 1', '41/18- 1', '41/18- 2', '41/20- 1', '41/25a- 1', '42/16- 1', '43/10- 1']


## Store secondary gr log in gr_table
Easy as loop through null wells invoking ***opendtect_to_psql.insert_log_as_arrays_query*** or ***opendtect_to_psql.insert_logs*** with the respective new log to search and populate. THe following cell uses the first. Note that a conflict statement must be provided in order to update the corresponding entry in a table.

In [41]:
cnfct = "UPDATE SET md_in_m = EXCLUDED.md_in_m, "
cnfct += "gr = EXCLUDED.gr, log_name = EXCLUDED.log_name"

for well in gr_null_wells:
    pp.execute_psql_command(
        op.insert_log_as_arrays_query(well, "Raw CDA Logs`GR [D]", "gr_table", "wells", conn, on_conflict_do=cnfct),
        conn
    )
    print("")

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Raw CDA Logs`GR [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`GR [D] not found for Well 20/06- 3.

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.217', 'Raw CDA Logs`GR [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`GR [D] not found for Well 20/06- 3Z.

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.221', 'Raw CDA Logs`GR [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`GR [D] not found for Well 21/26- 1D.

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.47', 'Raw CDA Logs`GR [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`GR [D] not found for Well 28/04a- 3.

Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_We

#### DTCO TABLE

In [8]:
# Null & empty wells in dtco table
dtco_null_wells, dtco_empty_wells = op.check_null_wells("dtco", "dtco_table", "wells", conn)


Well '20/06- 3' logs:
****************************************
Checkshots`One-way time 1[active]

Well '20/06- 3Z' logs:
****************************************
Checkshots`One-way time 1[active]

Well '29/06b- 5' logs:
****************************************
Checkshots`One-way time 1[active]

Well '37/10- 1' logs:
****************************************
Joined Well Logs`RHOB
Joined Well Logs`Deep Resistivity`ILD
Joined Well Logs`Deep Resistivity`RILD
Joined Well Logs`SP
Joined Well Logs`Micro Resistivity`MLL
Joined Well Logs`CALI
Joined Well Logs`DRHO
Joined Well Logs`GR
Joined Well Logs`MD ft
Joined Well Logs`Input to Derived Logs`SNP
Joined Well Logs`NPHI [D]
Joined Well Logs`Shallow Resistivity`SN

Well '37/23- 1' logs:
****************************************
Joined Well Logs`SP
Joined Well Logs`Deep Resistivity`RILD
Joined Well Logs`Shallow Resistivity`LLS
Joined Well Logs`Shallow Resistivity`SN
Joined Well Logs`CALI
Joined Well Logs`GR
Joined Well Logs`MD ft

Well '37/25- 1' 

In [7]:
# On conflcit statement
cnfct = "UPDATE SET md_in_m = EXCLUDED.md_in_m, "
cnfct += "dtco = EXCLUDED.dtco, log_name = EXCLUDED.log_name"
# Population loop
op.insert_logs(dtco_null_wells, "Raw CDA Logs`DT [D]", "dtco_table", "wells", conn, on_conflict_do=cnfct)

Proccessing insertion query. Concept: well log 'Raw CDA Logs`DT [D]' insertion in array mode

Well 20/06- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Raw CDA Logs`DT [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`DT [D] not found for Well 20/06- 3.

Well 20/06- 3Z
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.217', 'Raw CDA Logs`DT [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`DT [D] not found for Well 20/06- 3Z.

Well 21/26- 1D
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.221', 'Raw CDA Logs`DT [D]']' returned non-zero exit status 1.
Log Raw CDA Logs`DT [D] not found for Well 21/26- 1D.

Well 28/04a- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.47', 'Raw CDA Logs`DT [D]']' returned non-zero

"\nLog 'Raw CDA Logs`DT [D]' insertion completed in 59.00417923927307s"

#### DTSH TABLE

In [12]:
dtsh_null_wells, dtsh_empty_wells = op.check_null_wells("dtsh", "dtsh_table", "wells", conn)


Well '19/15- 1' logs:
****************************************
Joined Well Logs`Resistivity`P16H_UNC
Joined Well Logs`Resistivity`P22H_UNC
Joined Well Logs`Resistivity`P28H_UNC
Joined Well Logs`Resistivity`P34H_UNC
Joined Well Logs`Resistivity`P40H_UNC
Joined Well Logs`GR
Joined Well Logs`DTCO [D]
Joined Well Logs`Input to Derived Logs`DT

Well '20/06- 1' logs:
****************************************
Joined Well Logs`RHOB
Joined Well Logs`Deep Resistivity`ILD
Joined Well Logs`Deep Resistivity`LLD
Joined Well Logs`SP
Joined Well Logs`Micro Resistivity`MSFL
Joined Well Logs`Micro Resistivity`SFLU
Joined Well Logs`CALI
Joined Well Logs`Shallow Resistivity`LLS
Joined Well Logs`DRHO
Joined Well Logs`GR
Joined Well Logs`Input to Derived Logs`DT
Joined Well Logs`Input to Derived Logs`NPHI
Joined Well Logs`Input to Derived Logs`PEF
Joined Well Logs`DTCO [D]
Joined Well Logs`NPHI [D]
Joined Well Logs`PEF [D]
Checkshots`One-way time 1[active]

Well '20/06- 2' logs:
****************************

In [11]:
# On conflcit statement
cnfct = "UPDATE SET md_in_m = EXCLUDED.md_in_m, "
cnfct += "dtsh = EXCLUDED.dtsh, log_name = EXCLUDED.log_name"
# Population loop
op.insert_logs(dtsh_null_wells, "Raw CDA Logs`DTSH [D]", "dtsh_table", "wells", conn, on_conflict_do=cnfct)

Proccessing insertion query. Concept: well log 'Raw CDA Logs`DTSH [D]' insertion in array mode

Well 19/15- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Raw CDA Logs`DTSH [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Raw CDA Logs`DTSH [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`DTSH [D] not found for Well 19/15- 1.

Well 20/06- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.7', 'Raw CDA Logs`DTSH [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.7', 'Raw CDA Logs`DTSH [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`DTSH [D] not found for Well 20/06- 1.

Well 20/06- 2
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.11', 'Raw CDA Logs`DTSH [D]']' returned non-zero exit status 1.


...

INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.209', 'Raw CDA Logs`DTSH [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`DTSH [D] not found for Well 43/10- 1.

Well 44/12- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.211', 'Raw CDA Logs`DTSH [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.211', 'Raw CDA Logs`DTSH [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`DTSH [D] not found for Well 44/12- 1.


"\nLog 'Raw CDA Logs`DTSH [D]' insertion completed in 221.59122490882874s"

#### NPHI TABLE

In [17]:
nphi_null_wells, nphi_empty_wells = op.check_null_wells("nphi", "nphi_table", "wells", conn)


Well '19/08- 1' logs:
****************************************
Joined Well Logs`Deep Resistivity`RD
Joined Well Logs`Shallow Resistivity`RS
Joined Well Logs`GR
Joined Well Logs`Input to Derived Logs`K
Joined Well Logs`Input to Derived Logs`DT
Joined Well Logs`Input to Derived Logs`TH
Joined Well Logs`Input to Derived Logs`UR
Joined Well Logs`Input to Derived Logs`DTS
Joined Well Logs`POTA [D]
Joined Well Logs`DTCO [D]
Joined Well Logs`THOR [D]
Joined Well Logs`URAN [D]
Joined Well Logs`DTSH [D]
Raw CDA Logs`Th:K
Raw CDA Logs`Th:U
Raw CDA Logs`U:K
Raw CDA Logs`Deep Resistivity`RD1
Raw CDA Logs`Shallow Resistivity`RS1
Raw CDA Logs`DTCO [D]
Raw CDA Logs`Input to Dervied Logs`DTC
Raw CDA Logs`Input to Dervied Logs`DTS
Raw CDA Logs`Input to Dervied Logs`GR1
Raw CDA Logs`Input to Dervied Logs`K+Th
Raw CDA Logs`Input to Dervied Logs`K
Raw CDA Logs`Input to Dervied Logs`TH
Raw CDA Logs`Input to Dervied Logs`UR
Raw CDA Logs`DTSH  [D]
Raw CDA Logs`GR  [D]
Raw CDA Logs`POTA [D]
Raw CDA Logs`THOR

In [18]:
# On conflcit statement
cnfct = "UPDATE SET md_in_m = EXCLUDED.md_in_m, "
cnfct += "nphi = EXCLUDED.nphi, log_name = EXCLUDED.log_name"
# Population loop
op.insert_logs(nphi_null_wells, "Raw CDA Logs`NPHI [D]", "nphi_table", "wells", conn, on_conflict_do=cnfct)

Proccessing insertion query. Concept: well log 'Raw CDA Logs`NPHI [D]' insertion in array mode

Well 19/08- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.3', 'Raw CDA Logs`NPHI [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.3', 'Raw CDA Logs`NPHI [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`NPHI [D] not found for Well 19/08- 1.

Well 19/15- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Raw CDA Logs`NPHI [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Raw CDA Logs`NPHI [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`NPHI [D] not found for Well 19/15- 1.

Well 20/06- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Raw CDA Logs`NPHI [D]']' returned non-zero exit status 1.


...

INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.207', 'Raw CDA Logs`NPHI [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`NPHI [D] not found for Well 43/08- 2.

Well 43/10- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.209', 'Raw CDA Logs`NPHI [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.209', 'Raw CDA Logs`NPHI [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`NPHI [D] not found for Well 43/10- 1.


"\nLog 'Raw CDA Logs`NPHI [D]' insertion completed in 115.28783941268921s"

#### RHOB TABLE

In [21]:
rhob_null_wells, rhob_empty_wells = op.check_null_wells("rhob", "rhob_table", "wells", conn)


Well '19/08- 1' logs:
****************************************
Joined Well Logs`Deep Resistivity`RD
Joined Well Logs`Shallow Resistivity`RS
Joined Well Logs`GR
Joined Well Logs`Input to Derived Logs`K
Joined Well Logs`Input to Derived Logs`DT
Joined Well Logs`Input to Derived Logs`TH
Joined Well Logs`Input to Derived Logs`UR
Joined Well Logs`Input to Derived Logs`DTS
Joined Well Logs`POTA [D]
Joined Well Logs`DTCO [D]
Joined Well Logs`THOR [D]
Joined Well Logs`URAN [D]
Joined Well Logs`DTSH [D]
Raw CDA Logs`Th:K
Raw CDA Logs`Th:U
Raw CDA Logs`U:K
Raw CDA Logs`Deep Resistivity`RD1
Raw CDA Logs`Shallow Resistivity`RS1
Raw CDA Logs`DTCO [D]
Raw CDA Logs`Input to Dervied Logs`DTC
Raw CDA Logs`Input to Dervied Logs`DTS
Raw CDA Logs`Input to Dervied Logs`GR1
Raw CDA Logs`Input to Dervied Logs`K+Th
Raw CDA Logs`Input to Dervied Logs`K
Raw CDA Logs`Input to Dervied Logs`TH
Raw CDA Logs`Input to Dervied Logs`UR
Raw CDA Logs`DTSH  [D]
Raw CDA Logs`GR  [D]
Raw CDA Logs`POTA [D]
Raw CDA Logs`THOR

In [20]:
# On conflcit statement
cnfct = "UPDATE SET md_in_m = EXCLUDED.md_in_m, "
cnfct += "rhob = EXCLUDED.rhob, log_name = EXCLUDED.log_name"
# Population loop
op.insert_logs(rhob_null_wells, "Raw CDA Logs`RHOB [D]", "rhob_table", "wells", conn, on_conflict_do=cnfct)

Proccessing insertion query. Concept: well log 'Raw CDA Logs`RHOB [D]' insertion in array mode

Well 19/08- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.3', 'Raw CDA Logs`RHOB [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.3', 'Raw CDA Logs`RHOB [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`RHOB [D] not found for Well 19/08- 1.

Well 19/15- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Raw CDA Logs`RHOB [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.5', 'Raw CDA Logs`RHOB [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`RHOB [D] not found for Well 19/15- 1.

Well 20/06- 3
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.215', 'Raw CDA Logs`RHOB [D]']' returned non-zero exit status 1.


...

INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.207', 'Raw CDA Logs`RHOB [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`RHOB [D] not found for Well 43/08- 2.

Well 43/10- 1
Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.209', 'Raw CDA Logs`RHOB [D]']' returned non-zero exit status 1.


INFO:odpy.common:Failed:  Command '['C:\\PROGRA~1\\OPENDT~1\\6683E8~1.0\\bin\\win64\\Release\\od_WellMan', '--read-log', '100050.209', 'Raw CDA Logs`RHOB [D]']' returned non-zero exit status 1.


Log Raw CDA Logs`RHOB [D] not found for Well 43/10- 1.


"\nLog 'Raw CDA Logs`RHOB [D]' insertion completed in 99.28678965568542s"

# Fetch and insert Markers
---
Seismic markers will be used to slice intervals from tables using PSQL WHERE statements. The following cell fetches from Opendtect all the marker labels in the survey and store them in a list for further processing.    
    
"Base Rotliegendes" is a artificial marker to ease data slicing! The marker corresponds with the next marker after "Top Rotliegendes". This normalization reduces the amount of variables to take in consideration while executing the slicing process.

In [7]:
# Fectch marker names from OPENDTECT
marker_names = []
print(f"Fetching well markers from Opendtect:")
for well in well_names:
    print(f"\nWell {well}")
    for marker in wm.getMarkers(well)[0]:
        if marker not in marker_names:
            marker_names += [marker]
marker_names

Fetching well markers from Opendtect:

Well 19/08- 1

Well 19/15- 1

Well 20/06- 1

Well 20/06- 2

Well 20/06- 3

Well 20/06- 3Z

Well 20/06- 6

Well 20/12- 1

Well 20/12- 2

Well 20/12- 3

Well 21/21- 1

Well 21/26- 1D

Well 26/04- 1

Well 26/07- 1

Well 26/08- 1

Well 26/12- 1

Well 26/14- 1

Well 27/03- 1

Well 27/10- 1

Well 28/02a- 2

Well 28/04a- 3

Well 28/12- 1

Well 28/15- 1

Well 29/06b- 5

Well 29/11a- 1

Well 29/12- 1

Well 29/16- 1

Well 29/18- 1

Well 29/19- 1A

Well 29/19- 2

Well 29/20- 1

Well 29/23- 1

Well 29/23b- 2

Well 29/24- 1

Well 29/27- 1

Well 30/21- 1

Well 30/23- 1

Well 30/27- 1

Well 30/28- 1

Well 30/28- 2

Well 30/29a- 3

Well 30/30- 3

Well 31/26a- 12

Well 31/26c- 13

Well 31/27- 1

Well 36/13- 1

Well 36/15- 1

Well 36/26- 1

Well 37/10- 1

Well 37/12- 1

Well 37/23- 1

Well 37/25- 1

Well 38/01- 1

Well 38/03- 1

Well 38/10- 1

Well 38/16- 1

Well 38/18- 1

Well 38/22- 1

Well 38/24- 1

Well 38/25- 1

Well 38/29- 1

Well 39/01- 2

Well 39/01b- 3

We

['Sea bed',
 'Top Chalk',
 'Top Cromer Knoll',
 'Top Humber',
 'Top Heron',
 'TD Driller',
 'Top Moray',
 'Top Zechstein',
 'Top Rotliegendes',
 'Base Rotliegendes',
 'Top Old Red Sandstone',
 'Top Montrose',
 'Top Pre-Permian',
 'Top Miocene',
 'Top Hordaland',
 'Top Oligocene',
 'Top Stronsay',
 'Top Bacton',
 'Top Carboniferous',
 'Top Westphalian',
 'Top Visean',
 'Top Silurian',
 'Top Lower Paleozoic',
 'Top Eocene',
 'Top Paleocene',
 'Top Westray',
 'Top Fladen',
 'Top Triassic Undiff',
 'Top Nordland',
 'Top Dunlin',
 'Top Pliocene',
 'Top Dev/Carb',
 'Top Rotliegend Volcanics',
 'Top Haisborough',
 'Top Yoredale',
 'Top Whin Sill',
 'Top Scremerston',
 'Top Fell Sandstone',
 'Top Lias',
 'Top Cementstone',
 'Top Penarth',
 'Top Millstone Grit',
 'Top Coal Measures',
 'Well tops`Top West Sole',
 'Well tops`Top Lias',
 'Well tops`Top Penarth',
 'Well tops`Top Haisborough',
 'Well tops`Top Bacton',
 'Top West Sole']

With the markers, the next step to create the markers table. Then, construct an insertion query for them. 
   
If the well doesn't have any markers, Opendtect will raise an error. On the other hand, ff the well contains markers but doesn't have specific markers the query will insert NULL as its value. The insertion query is constructed automatically by ***opendtect_to_psql.insert_markers_query*** function.

In [10]:
marker_columns = ["well_name VARCHAR(30) UNIQUE NOT NULL REFERENCES wells(well_name),"]
for marker in marker_names:
    if marker != marker_names[-1]:
        marker_columns += [f"{pp.string_replacement(marker)} NUMERIC(7,2),"]
    if marker == marker_names[-1]:
        marker_columns += [f"{pp.string_replacement(marker)} NUMERIC(7,2)"]
marker_columns
pp.wells_table_creation("markers", conn, column_list=marker_columns)

'Query has been executed successfully in 0.06986522674560547s'

In [4]:
print("Proccessing insertion query. Concept: well markers")
for well in well_names:
    print(f"\nWwell {well}")
    pp.execute_psql_command(op.insert_markers_query(well, "markers"), conn)

Proccessing insertion query. Concept: well markers

Wwell 19/08- 1

Wwell 19/15- 1

Wwell 20/06- 1

Wwell 20/06- 2

Wwell 20/06- 3

Wwell 20/06- 3Z

Wwell 20/06- 6

Wwell 20/12- 1

Wwell 20/12- 2

Wwell 20/12- 3

Wwell 21/21- 1

Wwell 21/26- 1D

Wwell 26/04- 1

Wwell 26/07- 1

Wwell 26/08- 1

Wwell 26/12- 1

Wwell 26/14- 1

Wwell 27/03- 1

Wwell 27/10- 1

Wwell 28/02a- 2

Wwell 28/04a- 3

Wwell 28/12- 1

Wwell 28/15- 1

Wwell 29/06b- 5

Wwell 29/11a- 1

Wwell 29/12- 1

Wwell 29/16- 1

Wwell 29/18- 1

Wwell 29/19- 1A

Wwell 29/19- 2

Wwell 29/20- 1

Wwell 29/23- 1

Wwell 29/23b- 2

Wwell 29/24- 1

Wwell 29/27- 1

Wwell 30/21- 1

Wwell 30/23- 1

Wwell 30/27- 1

Wwell 30/28- 1

Wwell 30/28- 2

Wwell 30/29a- 3

Wwell 30/30- 3

Wwell 31/26a- 12

Wwell 31/26c- 13

Wwell 31/27- 1

Wwell 36/13- 1

Wwell 36/15- 1

Wwell 36/26- 1

Wwell 37/10- 1

Wwell 37/12- 1

Wwell 37/23- 1

Wwell 37/25- 1

Wwell 38/01- 1

Wwell 38/03- 1

Wwell 38/10- 1

Wwell 38/16- 1

Wwell 38/18- 1

Wwell 38/22- 1

Wwell 3

# Pandas DataFrames
---
Either structure data intro matrix like containers or use this format data to populate data into tables

## Lithostratigraphy
Opendtect's survey data lacks of lithostratigraphy information thus the fetching strategy changes. The data remains in the UK data repository in Excel tables. The fetching workflow goes as follows:
1. Manually migrate online data into csv files.
2. Amend and adapt csv file's content in order to be openned by Pandas DataFrames and accepted by PSQL.
3. Store each well's data in DataFrames as arrays.
4. Insert DataFrame's data into PSQL tables as arrays too.

### 2) Amend and adapt csv file's
#### Lithostratigraphy DataFrame's columns 
Due to how the data organized in the csv files, provide to DataFrames their column names will ease the creation and manipulation of objects. Includes string, numeric and feet columns for further processing as is shown in the following cell.   
    
Most of the data is in *feet*. Therefore is convertion to *meters* is a must. This will be done vectorially rather thab by samples.

In [7]:
# Dataframes columns
df_columns = [
    "Top_Down_Hole_Depth_m", 
    "Top_Vertical_Depth_MSL_m", 
    "Top_TWT_ms", 
    "Thickness", 
    "Upper_Litho", 
    "Lower_Litho", 
    "Upper_Chrono",
    "Upper_Chrono_ma",
    "Lower_Chrono",
    "Lower_Chrono_ma",
    "Litho_1",
    "pct_Litho_1",
    "Litho_2",
    "pct_Litho_2",
    "Litho_3",
    "pct_Litho_3",
    "Litho_4",
    "pct_Litho_4",
    "Cored",
    "Nature_of_Top",
    "HC_Present",
    "Notes"
]
str_columns = [
    "Upper_Litho", 
    "Lower_Litho", 
    "Upper_Chrono",
    "Lower_Chrono",
    "Litho_1",
    "Litho_2",
    "Litho_3",
    "Litho_4",
    "Cored",
    "Nature_of_Top",
    "HC_Present",
    "Notes"
]

numeric_columns = [
    "Top_Down_Hole_Depth_m", 
    "Top_Vertical_Depth_MSL_m", 
    "Top_TWT_ms", 
    "Thickness", 
    "Upper_Chrono_ma",
    "Lower_Chrono_ma",
    "pct_Litho_1",
    "pct_Litho_2",
    "pct_Litho_3",
    "pct_Litho_4"
]

feet_columns = [
    "Top_Down_Hole_Depth_m", 
    "Top_Vertical_Depth_MSL_m", 
    "Thickness", 
]

cols = [df_columns, str_columns, numeric_columns, feet_columns]
print("df_columns has the same length as numeric + str columns: ",
      len(df_columns) == len(numeric_columns) + len(str_columns)
)

df_columns has the same length as numeric + str columns:  True


#### Mending and formating process
The function ***py_to_psql.csv_to_df*** imports csv files into DataFrames. Refer to the docstring for more information about the corrections mentions so far.

In [13]:
# Set the working directory
wd_path = os.getcwd() + os.path.join("\\Wells_litho_from_website")
csv_path = wd_path + os.path.join("\\html")
# Store file names process them in a loop
wells_with_litho = os.listdir(csv_path)
# Loop to amend and format DataFrame data
print("Proccessing .csv files mending process")
for litho_csv in wells_with_litho:
    litho_csv_path = csv_path + os.path.join(f"\\{litho_csv}")
    c_litho_df_from_csv = pp.csv_to_df(litho_csv_path, columns=cols)
    # Better store the corrected dfs to new path as csvs (just in case)
    c_csv_path = wd_path + os.path.join(f"\\{litho_csv}")
    c_litho_df_from_csv.to_csv(c_csv_path, index=False)
    print(f"\nWell {litho_csv}")

Proccessing .csv files mending process

Well 20_06- 1.csv

Well 20_06- 2.csv

Well 20_12- 1.csv

Well 20_12- 2.csv

Well 20_12- 3.csv

Well 21_21- 1.csv

Well 21_26- 1D.csv

Well 26_07- 1.csv

Well 26_08- 1.csv

Well 26_12- 1.csv

Well 26_14- 1.csv

Well 27_03- 1.csv

Well 27_10- 1.csv

Well 28_12- 1.csv

Well 29_06b- 5.csv

Well 29_11a- 1.csv


C:\Users\Monster\AppData\Local\Temp/ipykernel_15368/1655034877.py:10: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  c_litho_df_from_csv = pp.csv_to_df(litho_csv_path, columns=cols)



Well 29_12- 1.csv

Well 29_16- 1.csv

Well 29_18- 1.csv

Well 29_19- 1A.csv

Well 29_19- 2.csv

Well 29_20- 1.csv

Well 29_23- 1.csv

Well 29_23b- 2.csv

Well 29_24- 1.csv

Well 29_27- 1.csv

Well 30_21- 1.csv

Well 30_23- 1.csv

Well 30_27- 1.csv

Well 30_28- 1.csv

Well 30_29a- 3.csv

Well 30_30- 3.csv

Well 31_26a- 12.csv

Well 31_27- 1.csv

Well 36_13- 1.csv

Well 36_15- 1.csv

Well 36_26- 1.csv

Well 37_10- 1.csv

Well 37_12- 1.csv

Well 37_23- 1.csv

Well 38_01- 1.csv

Well 38_03- 1.csv

Well 38_16- 1.csv

Well 38_18- 1.csv

Well 38_22- 1.csv

Well 38_24- 1.csv

Well 38_25- 1.csv

Well 38_29- 1.csv

Well 39_01- 2.csv

Well 39_02- 1.csv

Well 39_02- 4.csv

Well 39_07- 1.csv

Well 39_11- 1.csv

Well 39_16- 1.csv

Well 41_01- 1.csv

Well 41_08- 2.csv

Well 41_10- 1.csv

Well 41_14- 1.csv

Well 41_15- 1.csv

Well 41_18- 1.csv

Well 41_20- 1.csv

Well 41_25a- 1.csv

Well 42_09- 1.csv

Well 42_10a- 1.csv

Well 42_13- 1.csv

Well 42_16- 1.csv

Well 42_22- 1.csv

Well 42_23- 1.csv

Well

### 4) Insert DataFrame's data into PSQL tables
#### Table creation

In [14]:
litho_columns = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "Top_Down_Hole_Depth_m NUMERIC(12,2)[],", 
    "Top_Vertical_Depth_MSL_m NUMERIC(12,2)[],", 
    "Top_TWT_ms NUMERIC(12,2)[],", 
    "Thickness NUMERIC(7,2)[],", 
    "Upper_Litho VARCHAR(6)[],", 
    "Lower_Litho VARCHAR(6)[],", 
    "Upper_Chrono VARCHAR(6)[],",
    "Upper_Chrono_ma NUMERIC(6,2)[],",
    "Lower_Chrono VARCHAR(6)[],",
    "Lower_Chrono_ma NUMERIC(6,2)[],",
    "Litho_1 VARCHAR(6)[],",
    "pct_Litho_1 NUMERIC(5,2)[],",
    "Litho_2 VARCHAR(6)[],",
    "pct_Litho_2 NUMERIC(5,2)[],",
    "Litho_3 VARCHAR(6)[],",
    "pct_Litho_3 NUMERIC(5,2)[],",
    "Litho_4 VARCHAR(6)[],",
    "pct_Litho_4 NUMERIC(5,2)[],",
    "Cored VARCHAR(2)[],",
    "Nature_of_Top VARCHAR(3)[],",
    "HC_Present VARCHAR(3)[],",
    "Notes VARCHAR(80)[]"
]
pp.wells_table_creation("litho_table", conn, column_list=litho_columns)

'Query has been executed successfully in 0.13324284553527832s'

#### Litoestratigraphy insertion
The function ***py_to_psql.df_cols_to_query*** cosntructs an insertion query to populate Dataframe's columns as arrays into a destined table.

In [17]:
print(f"Proccessing insertion query. Concept: well's lithoestratigraphy")
for well in wells_with_litho:
    print(f"\nWell {well}")
    well_name = well.replace(".csv", "")
    #Df creation
    df = pp.csv_to_df(wd_path + os.path.join(f"\\{well}"), feet=False, columns=cols)
    #Query constructor
    insertion_query = pp.df_cols_to_query(df, "litho_table", well_name, conn)
    pp.execute_psql_command(insertion_query, conn)

Proccessing insertion query. Concept: well's lithoestratigraphy

Well 20_06- 1.csv

Well 20_06- 2.csv

Well 20_12- 1.csv

Well 20_12- 2.csv

Well 20_12- 3.csv


C:\Users\Monster\AppData\Local\Temp/ipykernel_15368/11337122.py:6: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  df = pp.csv_to_df(wd_path + os.path.join(f"\\{well}"), feet=False, columns=cols)



Well 21_21- 1.csv

Well 21_26- 1D.csv

Well 26_07- 1.csv

Well 26_08- 1.csv

Well 26_12- 1.csv

Well 26_14- 1.csv

Well 27_03- 1.csv

Well 27_10- 1.csv

Well 28_12- 1.csv

Well 29_06b- 5.csv

Well 29_11a- 1.csv

Well 29_12- 1.csv

Well 29_16- 1.csv

Well 29_18- 1.csv

Well 29_19- 1A.csv

Well 29_19- 2.csv

Well 29_20- 1.csv

Well 29_23- 1.csv

Well 29_23b- 2.csv

Well 29_24- 1.csv

Well 29_27- 1.csv

Well 30_21- 1.csv

Well 30_23- 1.csv

Well 30_27- 1.csv

Well 30_28- 1.csv

Well 30_29a- 3.csv

Well 30_30- 3.csv

Well 31_26a- 12.csv

Well 31_27- 1.csv

Well 36_13- 1.csv

Well 36_15- 1.csv

Well 36_26- 1.csv

Well 37_10- 1.csv

Well 37_12- 1.csv

Well 37_23- 1.csv

Well 38_01- 1.csv

Well 38_03- 1.csv

Well 38_16- 1.csv

Well 38_18- 1.csv

Well 38_22- 1.csv

Well 38_24- 1.csv

Well 38_25- 1.csv

Well 38_29- 1.csv

Well 39_01- 2.csv

Well 39_02- 1.csv

Well 39_02- 4.csv

Well 39_07- 1.csv

Well 39_11- 1.csv

Well 39_16- 1.csv

Well 41_01- 1.csv

Well 41_08- 2.csv

Well 41_10- 1.csv

Wel

## TectoB_table
TectoB table contains booleans to indicate whether well's logs sample the selected interval: the tectonostratigraphic B unit which is compounded by Permian Zechstein and Rotliegendes Groups.   
    
Is created by migrating a csv file to PSQL by samples. The populating process is similar to the lithostratigraphy section, but instead of importing data as arrays this one is done by samples using the ***py_to_psql.df_rows_to_query*** function.

In [4]:
#create the table
tectob_cols = [
    "well_name VARCHAR(30) NOT NULL UNIQUE REFERENCES wells(well_name),",
    "checkshot BOOLEAN,",
    "dtco BOOLEAN,",
    "dtsh BOOLEAN,",
    "gr BOOLEAN,",
    "nphi BOOLEAN,",
    "rhob BOOLEAN,",
    "litho BOOLEAN,",
    "notes TEXT"
]
pp.wells_table_creation("tectob_table", conn, column_list=tectob_cols)

'Query has been executed successfully in 0.11488628387451172s'

In [9]:
# Set working directory $ file
csv_table_path = os.getcwd() + os.path.join("\\csv_tables")
tectob_table_path = csv_table_path + os.path.join("\\tectb_table.csv")
# Create DataFrame
df = pp.csv_to_df(tectob_table_path, feet=False)
# Fill null values with numpy.nans
df = df.fillna(value=np.nan)
# Populating DataFrame's data by rows
insertion_query = pp.df_rows_to_query(df, "tectob_table", conn)
pp.execute_psql_command(insertion_query, conn)

C:\Users\Monster\AppData\Local\Temp/ipykernel_10484/1913381961.py:5: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only
  df = pp.csv_to_df(tectob_table_path, feet=False)


'Query has been executed successfully in 0.12035083770751953s'

Visualization
---
Every table can be displayed as DataFrames. It suffies with fetching PSQL data from tables and assign the first list as the DataFrame's columns and the second list as its values. The following cells illustrate some of the tables created before:
1. Overall well info
2. Gr table
3. Tectonostratigraphy B table   
4. Markers    
    
Note the lists in md_in_m and gr columns (gr table). Psycopg2 translates PSQL arrays into lists. In any case, note that by using arrays instead of samples the organization of data can be improved. If samples where used to store data, there would be an overload of repetitive data such as the well and log name by rows (knowing the table would contain more than 10.000.000 rows) or there would be 98 different gr tables, one for each well. Note that gr_df also exhibits 3.4+ kb of memory usage, hence arrays also guarantees the reduction of memory consumption.

In [12]:
# Query
wells_info_query = "SELECT * FROM wells"
wells_info_query_result = pp.fetch_psql_command(wells_info_query, conn)
# DataFrame creation and deployment
wells_info_df = pd.DataFrame(
    data=wells_info_query_result[1],
    columns=wells_info_query_result[0]
)
wells_info_df.head()

,well_id,opendtect_id,well_name,x_coordinate,y_coordinate,status
0,1,100050.3,19/08- 1,236383.46,6405003.51,OK
1,2,100050.5,19/15- 1,258036.61,6397675.39,OK
2,3,100050.7,20/06- 1,266301.42,6411338.06,OK
3,4,100050.1,20/06- 2,268517.68,6414712.53,OK
4,5,100050.2,20/06- 3,263640.71,6414053.21,OK


In [17]:
# Query
gr_query = "SELECT * FROM gr_table"
gr_query_result = pp.fetch_psql_command(gr_query, conn)
# DataFrame creation and deployment
gr_df = pd.DataFrame(
    data=gr_query_result[1],
    columns=gr_query_result[0]
)
gr_df.head()

,well_name,md_in_m,gr,log_name
0,19/08- 1,"[66.90360, 67.05600, 67.20840, 67.36080, 67.51...","[49.33200, 50.60500, 50.76100, 50.06900, 49.50...",Joined Well Logs`GR
1,19/15- 1,"[131.67360, 131.82600, 131.97839, 132.13080, 1...","[42.26000, 50.71000, 59.16000, 59.16000, None,...",Joined Well Logs`GR
2,20/06- 1,"[75.59040, 75.74280, 75.89520, 76.04760, 76.20...","[7.61720, 7.60160, 7.89450, 8.19530, 9.47660, ...",Joined Well Logs`GR
3,20/06- 2,"[115.67160, 115.82400, 115.97640, 116.12880, 1...","[8.59730, 7.84120, 7.51470, 7.96530, 8.90370, ...",Joined Well Logs`GR
4,20/06- 6,"[39.01440, 39.16680, 39.31920, 39.47160, 39.62...","[51.68490, 57.76560, 52.32500, 53.28510, 62.72...",Joined Well Logs`GR


In [19]:
gr_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   well_name  98 non-null     object
 1   md_in_m    79 non-null     object
 2   gr         79 non-null     object
 3   log_name   79 non-null     object
dtypes: object(4)
memory usage: 3.2+ KB


In [20]:
# Query
tectob_query = "SELECT * FROM tectob_table"
tectob_result = pp.fetch_psql_command(tectob_query, conn)
# DataFrame creation and deployment
tectob_df = pd.DataFrame(
    data=tectob_result[1],
    columns=tectob_result[0]
)
tectob_df.tail()

,well_name,checkshot,dtco,dtsh,gr,nphi,rhob,litho,notes
93,43/06- 1,None,None,None,None,None,None,None,None
94,43/07- 1,None,None,None,None,None,None,None,None
95,43/08- 2,None,None,None,None,None,None,None,None
96,43/10- 1,None,None,None,None,None,None,None,None
97,44/12- 1,None,True,None,True,True,True,True,Gran Bache a nivel de tope Zechstein en RHOB y...


In [24]:
# Query
marker_query = """
SELECT well_name, top_zechstein, top_rotliegendes, base_rotliegendes
FROM markers
WHERE (top_zechstein, top_rotliegendes, base_rotliegendes) IS NOT NULL
"""
marker_query_result = pp.fetch_psql_command(marker_query, conn)
# DataFrame creation and deployment
marker_df = pd.DataFrame(
    data=marker_query_result[1],
    columns=marker_query_result[0]
)
marker_df

,well_name,top_zechstein,top_rotliegendes,base_rotliegendes
0,20/06- 1,2403.35,2416.15,2449.07
1,20/12- 2,3483.86,3820.36,3995.01
2,20/12- 3,1058.88,1195.43,1257.60
3,26/07- 1,442.50,1133.00,1285.00
4,26/08- 1,786.38,1480.41,1870.25
5,26/14- 1,512.06,1127.76,1136.29
6,28/12- 1,1165.56,2187.24,2206.75
7,29/16- 1,1502.05,2952.90,3261.36
8,29/18- 1,2271.67,3210.15,3734.71
9,29/20- 1,2472.54,2630.12,2788.92


base_rotliegend is a dummy marker that ease data filtering. base_rotliegend is the normalization of any of the markers following top_rotliegendes. It is much simpler to use a single variable as the lower bound of the interval than 5 (being: old red sandstone, Carboniferous, Yoredale, Scremerston, TD Driller, etc).

# Unnesting and slicing data
---
As it was mention before, log data is organized by arrays. Arrays preclude most of the action over the data such as vizualization or statistical analysis. To prepare the data for further analysis, the array organization will be changed to sample organization by unnesting PSQL arrays. After the unnesting query, these will be filtered by fetching logs between the selected interval due to the massive quantity of samples.   
   
The process of unnesting and slicing is done by looping through the wells and making queries for each one of these. The following cell illustrates the process of constructing the queries with the ***py_to_psql.slice_unnest_data_query*** function and the population of the corresponding data into DataFrames. Do note that, for each row, there is a sample and a repetitive field that could potencially overload memory consumption if the logs weren't sliced. The output of the process consist of a DataFrame with more than 250.000 samples that although it contains less data than *gr_df*, it usages 16.3 mb of memory. This fact verifies that storing all the logs for all the wells in the survey using a sample organization is not memory wise.

In [25]:
# Empty dataframe
log_table = ["well_name","md_in_m","gr"]
sliced_logs_df = pd.DataFrame(columns=log_table)
# loop through markers df and fetch: wells and top-base of interval
for row in marker_df.values:
    # Query for: unnest & slice logs
    filtered_unnested_query = pp.slice_unnest_data_query(
        row[0],
        log_table,
        "gr_table",
        "markers",
        marker_df.columns[1],
        row[1],
        marker_df.columns[3],
        row[3],
        md_column_name=log_table[1]
    )
    # Fetch data
    query_result = pp.fetch_psql_command(filtered_unnested_query, conn)
    # Construct DataFrame with all samples related to one well
    temp_df = pd.DataFrame(
        data=query_result[1], 
        columns=sliced_logs_df.columns
    )
    # truncate the 5th decimal of float columns
    temp_df[["md_in_m", "gr"]] = np.floor(temp_df[["md_in_m", "gr"]].astype("float").round(5)*10000)/10000
    # Append all samples related to one well into main DataFrame
    sliced_logs_df = sliced_logs_df.append(temp_df, ignore_index=True)
sliced_logs_df.head()

,well_name,md_in_m,gr
0,20/06- 1,2403.5004,13.9245
1,20/06- 1,2403.6528,19.3998
2,20/06- 1,2403.8051,23.8073
3,20/06- 1,2403.9575,25.6146
4,20/06- 1,2404.1101,26.8944


In [26]:
sliced_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 169990 entries, 0 to 169989
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   well_name  169990 non-null  object 
 1   md_in_m    169990 non-null  float64
 2   gr         161606 non-null  float64
dtypes: float64(2), object(1)
memory usage: 3.9+ MB


The ***py_to_psql.unnested_logs_to_df*** function automatizes the unnesting, slicing and storation of data into DataFrames. The final step is to append all the sampled logs into one DataFrame. The merging process uses PSQL full join method to organize measurements by well name and depth (md_in_m) so that all the logs are tied to the same vertical axis. Nans mean that there is no value in a specific log and depth.

In [27]:
foo = ["dtco", "dtsh", "rhob", "nphi"]
for log in foo:
    sliced_logs_df = sliced_logs_df.merge(
        pp.unnested_logs_to_df(marker_df,"well_name","md_in_m",f"{log}",f"{log}_table","markers", conn),
        how="outer", 
        on=["well_name", "md_in_m"]
    )
sliced_logs_df["md_round2"] = sliced_logs_df["md_in_m"].round(2)
sliced_logs_df

,well_name,md_in_m,gr,dtco,dtsh,rhob,nphi,md_round2
0,20/06- 1,2403.5004,13.9245,49.7206,NaN,2.8662,0.0044,2403.50
1,20/06- 1,2403.6528,19.3998,49.1776,NaN,2.9032,0.0053,2403.65
2,20/06- 1,2403.8051,23.8073,50.0415,NaN,2.9078,0.0076,2403.81
3,20/06- 1,2403.9575,25.6146,50.7914,NaN,2.8997,0.0091,2403.96
4,20/06- 1,2404.1101,26.8944,51.1254,NaN,2.8898,0.0099,2404.11
...,...,...,...,...,...,...,...,...
171272,29/27- 1,2896.2597,NaN,NaN,NaN,NaN,0.1703,2896.26
171273,29/27- 1,2896.3105,NaN,NaN,NaN,NaN,0.1708,2896.31
171274,29/27- 1,2896.3613,NaN,NaN,NaN,NaN,0.1713,2896.36
171275,29/27- 1,2896.4121,NaN,NaN,NaN,NaN,0.1701,2896.41


In [28]:
sliced_logs_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 171277 entries, 0 to 171276
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   well_name  171277 non-null  object 
 1   md_in_m    171277 non-null  float64
 2   gr         161606 non-null  float64
 3   dtco       154489 non-null  float64
 4   dtsh       2617 non-null    float64
 5   rhob       102262 non-null  float64
 6   nphi       98756 non-null   float64
 7   md_round2  171277 non-null  float64
dtypes: float64(7), object(1)
memory usage: 11.8+ MB


In [29]:
print(sliced_logs_df.head(25))

   well_name    md_in_m       gr     dtco  dtsh    rhob    nphi  md_round2
0   20/06- 1  2403.5004  13.9245  49.7206   NaN  2.8662  0.0044    2403.50
1   20/06- 1  2403.6528  19.3998  49.1776   NaN  2.9032  0.0053    2403.65
2   20/06- 1  2403.8051  23.8073  50.0415   NaN  2.9078  0.0076    2403.81
3   20/06- 1  2403.9575  25.6146  50.7914   NaN  2.8997  0.0091    2403.96
4   20/06- 1  2404.1101  26.8944  51.1254   NaN  2.8898  0.0099    2404.11
5   20/06- 1  2404.2624  29.8385  52.1258   NaN  2.8859  0.0125    2404.26
6   20/06- 1  2404.4147  33.0103  54.5925   NaN  2.8697  0.0258    2404.41
7   20/06- 1  2404.5671  34.5103  55.9905   NaN  2.8258  0.0538    2404.57
8   20/06- 1  2404.7194  37.7556  57.7932   NaN  2.7677  0.0913    2404.72
9   20/06- 1  2404.8720  43.0833  65.3615   NaN  2.6521  0.1446    2404.87
10  20/06- 1  2405.0244  52.4583  71.7067   NaN  2.5152  0.1955    2405.02
11  20/06- 1  2405.1767  57.0625  74.1055   NaN  2.4832  0.2335    2405.18
12  20/06- 1  2405.3291  

## Append litoestratigraphy data to unnested logs df
### Fetch lithostratigraphy data 

In [30]:
litho_columns = [
    "well_name", 
    "top_down_hole_depth_m", 
    "upper_litho", 
    "upper_chrono", 
    "litho_1", 
    "pct_litho_1", 
    "litho_2", 
    "pct_litho_2", 
    "litho_3", 
    "pct_litho_3",
    "litho_4", 
    "pct_litho_4",
    "hc_present"
]
# Empty dataframe
sliced_litho_df = pd.DataFrame(columns=litho_columns)
# loop through markers df and fetch: wells and top-base of interval
for row in marker_df.values:
    # Query for: unnest & slice litho
    filtered_unnested_query =pp.slice_unnest_data_query(
        row[0],
        litho_columns,
        "litho_table",
        "markers",
        marker_df.columns[1],
        row[1],
        marker_df.columns[3],
        row[3],
        md_column_name=litho_columns[1]
    )
    # Fetch data
    query_result = pp.fetch_psql_command(filtered_unnested_query, conn)
    # Construct DataFrame with all samples related to one well
    temp_df = pd.DataFrame(
        data=query_result[1], 
        columns=sliced_litho_df.columns
    )
    # Append all samples related to one well into main DataFrame
    sliced_litho_df = sliced_litho_df.append(temp_df, ignore_index=True)
sliced_litho_df.head()

,well_name,top_down_hole_depth_m,upper_litho,upper_chrono,litho_1,pct_litho_1,litho_2,pct_litho_2,litho_3,pct_litho_3,litho_4,pct_litho_4,hc_present
0,20/06- 1,2403.35,TURB,P,ANHY,80.00,SHAL,10.00,SLST,10.00,None,0.00,None
1,20/06- 1,2406.70,KUPF,P,ANHY,80.00,SHAL,20.00,None,0.00,None,0.00,None
2,20/06- 1,2416.15,AUKF,P,SDST,95.00,SHAL,5.00,None,0.00,None,0.00,None
3,20/06- 1,2449.07,*,CL,SDST,35.00,CLST,30.00,LMST,25.00,SLST,10.00,None
4,20/12- 2,3483.86,ZECH,PU,HALI,80.00,ANHY,15.00,DOLO,5.00,None,0.00,None


In [31]:
sliced_litho_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   well_name              242 non-null    object
 1   top_down_hole_depth_m  242 non-null    object
 2   upper_litho            241 non-null    object
 3   upper_chrono           242 non-null    object
 4   litho_1                227 non-null    object
 5   pct_litho_1            242 non-null    object
 6   litho_2                139 non-null    object
 7   pct_litho_2            242 non-null    object
 8   litho_3                66 non-null     object
 9   pct_litho_3            242 non-null    object
 10  litho_4                18 non-null     object
 11  pct_litho_4            242 non-null    object
 12  hc_present             23 non-null     object
dtypes: object(13)
memory usage: 24.7+ KB


## Merging logs & lithostratigraphy DataFrames
To improve further insights, the lithostratigraphic data will be joinned with log and marker data into one Dataframe.   
    
These two Dataframes can not be merged conventionally due to few differences in the vertical axis triggered by different sampling. A different aproach must be applied, therefore Pandas.merge_asof will be used to surpass the difference in axis considering the following:
1. Pandas.merge_asof works only if the merging/joining axis is sorted. Both logs and litho dataframes are structured in two dimensions, i.e., they are controlled by well_name and md thus they cannot be sorted only by md. A loop through wells will be used to reduce the dimension of Dataframes so that merge_asof can be applied.
2. There can only be one join per sample in both Dataframes and lithostratigraphic samples must be paired to nearest or exact values of depth.

In [32]:
# Create a common column to join Dataframes
sliced_logs_df["md_round2"] = sliced_logs_df["md_in_m"].round(2)
sliced_litho_df["md_round2"] = sliced_litho_df["top_down_hole_depth_m"].astype("float").round(2)
# Empty DataFrame to append data from well loop
right_join_indexes_df = pd.DataFrame()
for well in marker_df["well_name"]:
    # Here a right join is done to identify the index of left nearest values rather than joining it directly into left.
    # The tolerance doesn't work on left join because the difference between markers and logs are not fixed differences,
    # and due to the massive Dataframe, a lot of values of left are joined to one value of right.
    # It does work on right join due to the lack of samples. Each sample of right is joined with one sample of left (nearest)
    temp_merged_df = pd.merge_asof(
        sliced_litho_df[sliced_litho_df["well_name"] == well],
        sliced_logs_df[sliced_logs_df["well_name"] == well].reset_index(),
        on="md_round2",
        direction="nearest",
        tolerance=0.5
    )
    right_join_indexes_df = right_join_indexes_df.append(temp_merged_df)
# Left join on index rather than md_round2
sliced_litho_logs_df = pd.merge(
        sliced_logs_df,
        right_join_indexes_df[[
            "index", "md_round2",
            "top_down_hole_depth_m",
            "hc_present",
            "litho_1", "pct_litho_1",
            "litho_2", "pct_litho_2",
            "litho_3", "pct_litho_3",
            "litho_4", "pct_litho_4"
        ]],
        how="left", 
        right_on="index",
        left_index=True
    ).set_index("index")

In [33]:
pd.set_option("display.max_rows", 300)
sliced_litho_logs_df[~sliced_litho_logs_df['litho_1'].isnull()]

,well_name,md_in_m,gr,dtco,dtsh,rhob,nphi,md_round2_x,md_round2_y,top_down_hole_depth_m,hc_present,litho_1,pct_litho_1,litho_2,pct_litho_2,litho_3,pct_litho_3,litho_4,pct_litho_4
index,,,,,,,,,,,,,,,,,,,
0.0,20/06- 1,2403.5004,13.9245,49.7206,NaN,2.8662,0.0044,2403.50,2403.35,2403.35,None,ANHY,80.00,SHAL,10.00,SLST,10.00,None,0.00
21.0,20/06- 1,2406.7006,21.8789,50.9886,NaN,2.9524,-0.0023,2406.70,2406.70,2406.70,None,ANHY,80.00,SHAL,20.00,None,0.00,None,0.00
83.0,20/06- 1,2416.1496,44.8507,69.8345,NaN,2.4702,0.2604,2416.15,2416.15,2416.15,None,SDST,95.00,SHAL,5.00,None,0.00,None,0.00
299.0,20/06- 1,2449.0681,108.1042,72.7872,NaN,2.4210,0.2159,2449.07,2449.07,2449.07,None,SDST,35.00,CLST,30.00,LMST,25.00,SLST,10.00
300.0,20/12- 2,3483.8640,14.3906,NaN,NaN,2.6817,NaN,3483.86,3483.86,3483.86,None,HALI,80.00,ANHY,15.00,DOLO,5.00,None,0.00
2508.0,20/12- 2,3820.3632,29.0625,NaN,NaN,2.7285,0.0556,3820.36,3820.36,3820.36,None,SDST,60.00,SLST,30.00,CLST,10.00,None,0.00
3653.0,20/12- 2,3994.8610,75.5000,65.8750,NaN,2.5618,0.1150,3994.86,3995.01,3995.01,None,SDST,40.00,SLST,30.00,CLST,30.00,None,0.00
3654.0,20/12- 3,1059.0275,14.2344,52.0000,NaN,2.9525,-0.0066,1059.03,1058.88,1058.88,None,ANHY,100.00,None,0.00,None,0.00,None,0.00
3755.0,20/12- 3,1074.4200,18.4061,67.3750,NaN,2.8005,0.1101,1074.42,1074.42,1074.42,None,ANHY,80.00,DOLO,20.00,None,0.00,None,0.00


In [34]:
# os.mkdir(os.getcwd() + os.path.join("\\Filtered_data"))
wd_path = os.getcwd() + os.path.join("\\Filtered_data")
sliced_litho_logs_df.to_csv(wd_path + os.path.join("\\sliced_litho_logs.csv"))
sliced_logs_df.to_csv(wd_path + os.path.join("\\sliced_logs.csv"))

In [35]:
sliced_logs_df

,well_name,md_in_m,gr,dtco,dtsh,rhob,nphi,md_round2
0,20/06- 1,2403.5004,13.9245,49.7206,NaN,2.8662,0.0044,2403.50
1,20/06- 1,2403.6528,19.3998,49.1776,NaN,2.9032,0.0053,2403.65
2,20/06- 1,2403.8051,23.8073,50.0415,NaN,2.9078,0.0076,2403.81
3,20/06- 1,2403.9575,25.6146,50.7914,NaN,2.8997,0.0091,2403.96
4,20/06- 1,2404.1101,26.8944,51.1254,NaN,2.8898,0.0099,2404.11
...,...,...,...,...,...,...,...,...
171272,29/27- 1,2896.2597,NaN,NaN,NaN,NaN,0.1703,2896.26
171273,29/27- 1,2896.3105,NaN,NaN,NaN,NaN,0.1708,2896.31
171274,29/27- 1,2896.3613,NaN,NaN,NaN,NaN,0.1713,2896.36
171275,29/27- 1,2896.4121,NaN,NaN,NaN,NaN,0.1701,2896.41
